# TensorTrack — example workflow

This notebook demonstrates a clean, user-friendly Python workflow we should aim for when refactoring the current `tensortrack` Python surface. It shows high-level API calls (create/apply/inspect/serialize) and short notes about edge cases and best practices.

## Goals shown here

- Minimal, idiomatic Python API for ML users (NumPy / PyTorch friendly).
- Simple round-trip: create a patch, serialize, deserialize, apply to a base tensor.
- Inspectable, versioned patch format with metadata and verification.

## High-level API (design sketch)

Module: `tensortrack`
- Core functions:
  - `create_patch(base, target, *, algorithm='delta', compress=True, metadata=None) -> Patch`
  - `apply_patch(base, patch, *, out=None) -> patched_tensor`
  - `Patch.from_bytes(b: bytes) -> Patch` and `Patch.to_bytes() -> bytes`
  - `inspect_patch(b: bytes) -> dict` (quick metadata + checksums estimate)
- Types: `Patch` (opaque object with `metadata`, `version`, helper methods), and domain errors in `tensortrack.errors`

Design notes: accept NumPy arrays and PyTorch tensors (copying to CPU if needed). `apply_patch` accepts either a `Patch` instance or raw bytes. Patches are versioned (e.g. `ttpatch/v1`).

In [1]:
# Example usage (NumPy + optional PyTorch).
import numpy as np
try:
    import torch
    has_torch = True
except Exception:
    has_torch = False

# The API we expect after refactor (high-level, blocking):
from tensortrack import create_patch, apply_patch, Patch

# Minimal NumPy round-trip example (illustrative - requires package implementation)
base = np.zeros((3, 3), dtype=np.float32)
target = np.eye(3, dtype=np.float32)

# The following calls assume the refactored API exists.
patch = create_patch(base, target, compress=True, metadata={"note": "example"})
print('patch size bytes:', len(patch.to_bytes()))
applied = apply_patch(base, patch)
assert np.array_equal(applied, target)
print('NumPy roundtrip successful')

# If PyTorch is available, show the intended pattern (device-aware):
if has_torch:
    t_base = torch.zeros(3, 3, dtype=torch.float32)
    t_target = torch.eye(3, dtype=torch.float32)
    patch2 = create_patch(t_base, t_target)
    out = apply_patch(t_base, patch2)
    print('torch roundtrip ok', torch.allclose(out, t_target))

ImportError: cannot import name 'create_patch' from 'tensortrack' (/Users/james/dev/tensortrack/.venv/lib/python3.9/site-packages/tensortrack/__init__.py)

## Serialization + verification pattern

- `b = patch.to_bytes()` — produces a compact, versioned binary with a small JSON metadata header and compressed payload.
- `Patch.from_bytes(b)` — reconstructs the Patch object, validates header and checksums.
- `patch.inspect()` — cheap read-only metadata (dtype, shape, base_checksum, target_checksum, estimated_payload_size).
- Verification on `apply_patch` should assert base checksum matches unless `force=True` is provided. Corrupted or mismatched patches raise `tensortrack.errors.VerificationError`.

## Edge cases and operational notes

- Dtype / shape mismatch: surface as explicit errors; do not silently cast. Provide optional conversion flags.
- Large tensors: provide streaming/chunked APIs and a `chunk_size` parameter in `create_patch` and `apply_patch`.
- Device handling: accept torch tensors on CUDA by copying to CPU for patch generation or offering device-aware fast paths. Document cost of device transfers.
- Endianness: encode endianness in header so patches are portable across architectures.

## Quick CLI examples (intended)
```bash
tensortrack create --base base.npy --target target.npy --out model.patch
tensortrack apply --base base.npy --patch model.patch --out patched.npy
tensortrack inspect --patch model.patch
```

## Next steps (use this notebook as a checkpoint)

1. Implement a small `Patch` Python wrapper around the existing Rust bytes/ops.
2. Add the `create_patch` / `apply_patch` thin Python helpers that accept NumPy/PyTorch.
3. Add unit tests that exercise the notebook examples (round-trip).